Itay Koren PANDA TASK

In [3]:
pip install psycopg2 -U

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [5]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-fc-barcelona-20-21-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [6]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)


squad table:

        player_id: player's id

        first_name: player's first name

        last_name: player's last name

        position: position in the field - gk,df,mf,fw

games_played table:

        player_id: player's id

        league: league appearances

        champions: champions appearances

        cup: cup appearances

        supercup: supercup appearances

league_stats table:

        player_id: player's id

        league_goals: goals in league

        league_assits: assits in league

        league_yellow: yellow cards in league

        league_red: red cards in league

        league_minutes: minutes played in league

champions_stats table:

        player_id: player's id

        champions_goals: goals in champions

        champions_assits: assits in champions

        champions_yellow: yellow cards in champions

        champions_red: red cards in champions

        champions_minutes: minutes played in champions

supercup_stats table:

        player_id: player's id

        supercup_goals: goals in supercup

        supercup_assits: assits in supercup

        supercup_yellow: yellow cards in supercup

        supercup_red: red cards in supercup

        supercup_minutes: minutes played in supercup


1. What is the position played by the majority of the squad?


In [13]:
# squad table:

# player_id: player's id

# first_name: player's first name

# last_name: player's last name

# position: position in the field - gk,df,mf,fw

query = '''
    SELECT  
        position,
        COUNT(position) as pos_count
    FROM
        squad
    GROUP BY
        position
    ORDER BY
        pos_count DESC
    LIMIT 1;
'''
queryResult(query)

,position,pos_count
0,df,9



2. Which players played the most games during the entire season?


In [62]:
# games_played table:

#     player_id: player's id

#     league: league appearances

#     champions: champions appearances

#     cup: cup appearances

#     supercup: supercup appearances

query = '''
    SELECT
        DISTINCT player_id,
        ((SUM(league)  OVER(PARTITION BY player_id))  + (SUM(champions)  OVER(PARTITION BY player_id))
        + (SUM(supercup)  OVER(PARTITION BY player_id)) + (SUM(cup)  OVER(PARTITION BY player_id))
        )  as total_appr
    FROM
        games_played
    ORDER BY
        total_appr DESC
    LIMIT 1
    ;
'''
queryResult(query)

,player_id,total_appr
0,19,52



3. Out of the players who played less than 1000 minutes in the league who scored the most goals?


In [72]:
# league_stats table:

#     player_id: player's id

#     league_goals: goals in league

#     league_assits: assits in league

#     league_yellow: yellow cards in league

#     league_red: red cards in league

#     league_minutes: minutes played in league

query = '''
    SELECT  
        player_id,
        league_goals
    FROM
        league_stats
    WHERE
        league_goals = (SELECT MAX(league_goals) FROM league_stats WHERE league_minutes < 1000)
'''
queryResult(query)

,player_id,league_goals
0,22,4



4. Which players played the most minutes in the league? Show top 5.


In [74]:
# league_stats table:

#     player_id: player's id

#     league_goals: goals in league

#     league_assits: assits in league

#     league_yellow: yellow cards in league

#     league_red: red cards in league

#     league_minutes: minutes played in league

query = '''
    SELECT  
    player_id,
    league_minutes
    FROM
        league_stats
    ORDER BY 
        league_minutes DESC
    LIMIT 5
'''
queryResult(query)

,player_id,league_minutes
0,15,3158
1,12,3030
2,29,3022
3,1,2790
4,28,2619



5.How many assists were made by the forwards?


In [97]:
# squad table:                                                        
# player_id: player's id                            
# position: position in the field - gk,df,mf,fw           
                                                             
# league_stats table:
# player_id: player's id
# league_assits: assits in league

# champions_stats table:
# player_id: player's id
# champions_assits: assits in champions

# supercup_stats table:
# player_id: player's id
# supercup_assits: assits in supercup



query = '''
    SELECT  
       squad.player_id,
       league_stats.league_assists + champions_stats.champions_assists 
       + supercup_stats.supercup_assists as forword_assists
    FROM
        squad
        INNER JOIN league_stats 
            ON league_stats.player_id = squad.player_id
        INNER JOIN champions_stats
            ON champions_stats.player_id = squad.player_id
        INNER JOIN supercup_stats
            ON supercup_stats.player_id = squad.player_id
    WHERE 
        squad.position = 'fw'
'''
queryResult(query)

,player_id,forword_assists
0,22,4
1,23,0
2,24,5
3,25,2
4,26,0
5,27,4
6,28,9
7,29,13



6. Who are the top 3 players with the best goals per minute record? (clarification: how many minutes pass between each goals. For example if a player scored 5 goals and played 600 minutes then his goals per minute record is a goal every 120 minutes)


In [135]:
# squad table:
#         player_id: player's id

# league_stats table:
#         player_id: player's id
#         league_goals: goals in league
#         league_minutes: minutes played in league

# champions_stats table:
#         player_id: player's id
#         champions_goals: goals in champions
#         champions_minutes: minutes played in champions

# supercup_stats table:
#         player_id: player's id
#         supercup_goals: goals in supercup
#         supercup_minutes: minutes played in supercup


query = '''
    SELECT  
        squad.player_id,
        league_stats.league_goals + champions_stats.champions_goals + supercup_stats.supercup_goals as total_goals,
        league_stats.league_minutes + champions_stats.champions_minutes + supercup_stats.supercup_minutes as total_min,
        (league_stats.league_minutes + champions_stats.champions_minutes + supercup_stats.supercup_minutes)/
        (league_stats.league_goals + champions_stats.champions_goals + supercup_stats.supercup_goals )
        AS goals_per_min
    FROM
        squad
        INNER JOIN league_stats 
            ON league_stats.player_id = squad.player_id
        INNER JOIN champions_stats
            ON champions_stats.player_id = squad.player_id
        INNER JOIN supercup_stats
            ON supercup_stats.player_id = squad.player_id
        WHERE
            league_stats.league_goals > 0
            OR champions_stats.champions_goals > 0
            OR supercup_stats.supercup_goals > 0
        ORDER BY
            goals_per_min ASC
        LIMIT 3
'''
queryResult(query)

,player_id,total_goals,total_min,goals_per_min
0,29,35,3682,105
1,22,5,596,119
2,28,17,3362,197



7. How many players did not contribute goals or assists in the league?


In [137]:
# squad table:
#         player_id: player's id

# league_stats table:
#         player_id: player's id
#         league_goals: goals in league
#         league_assits: assits in league


# champions_stats table:
#         player_id: player's id
#         champions_goals: goals in champions
#         champions_assits: assits in champions

# supercup_stats table:
#         player_id: player's id
#         supercup_goals: goals in supercup
#         supercup_assits: assits in supercup



query = '''
    SELECT  
        COUNT(league_stats.player_id)
    FROM
        league_stats
    WHERE
        league_goals = 0
        and
        league_assists = 0

'''
queryResult(query)

,count
0,10



8. Who is the player that has the most yellow cards in the league? (If there is a tie then sort by minutes played)


In [141]:
query = '''
    SELECT  
        *
    FROM
        league_stats
    ORDER BY
        league_yellow DESC
    LIMIT 1;
'''
queryResult(query)

,player_id,league_goals,league_assists,league_yellow,league_red,league_minutes
0,12,3,7,9,0,3030



9. Which defender contribute the most goals and assists in the league?


In [157]:

query = '''
    SELECT  
    *
    FROM
     (SELECT
                league_goals,
                league_assists,
               league_goals + league_assists as total_g_a
            FROM
                league_stats
        ) as subq,
        squad
    WHERE
        squad.position = 'df'
    ORDER BY
        subq.total_g_a DESC
    LIMIT 1;
'''
queryResult(query)

,league_goals,league_assists,total_g_a,player_id,first_name,last_name,position
0,30,11,41,5,Junior,Firpo,df



10. How many players played more then 50 hours total in the league and in the champions league?

In [164]:
;query = '''
    SELECT  
        COUNT(subq.player_id)
    FROM
        (SELECT
            champions_stats.player_id,
            (league_stats.league_minutes + champions_stats.champions_minutes) / 60 as total_hour
        FROM
            champions_stats,
            league_stats
        ) as subq
    WHERE
        subq.total_hour > 50 
    

'''
queryResult(query)

,count
0,102
